In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.01
schedule = [200, 500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_pggan/cutmix4' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)
random_crop = (144, 144)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 3000]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.8
cm_prob_low = 0.5

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_style1')
# source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(random_crop),
    transforms.RandomCrop(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(random_crop),
    transforms.CenterCrop(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
# source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
#                                 batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader,teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()

    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
#         source_inputs, source_targets = source_set[batch_idx]
#         source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        targets_a = targets
        targets_b = targets[rand_index]
        
        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        
        lam_ratio = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        targets_a = targets
        targets_b = targets[rand_index]
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets_a)*lam_ratio + criterion(outputs, targets_b)*(1 - lam_ratio)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
             batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
34/34 Data:0.000 | Batch:0.370 | Total:0:00:20 | ETA:0:00:01 | Loss:12.913346648216248 | top1:53.787879943847656


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.000 | Batch:0.763 | Total:0:00:44 | ETA:0:00:00 | Loss:22.221639793609903 | top1:48.769474029541016
39/39 Data:0.206 | Batch:0.442 | Total:0:00:19 | ETA:0:00:00 | Loss:22.18970572642791 | top1:50.25640869140625

Epoch: [2 | 4000] LR: 0.010600
34/34 Data:0.000 | Batch:0.432 | Total:0:00:18 | ETA:0:00:01 | Loss:17.89129338119969 | top1:50.0

Epoch: [3 | 4000] LR: 0.011200
34/34 Data:0.000 | Batch:0.497 | Total:0:00:13 | ETA:0:00:01 | Loss:9.023871436263576 | top1:53.030303955078125

Epoch: [4 | 4000] LR: 0.011800
34/34 Data:0.000 | Batch:0.493 | Total:0:00:15 | ETA:0:00:01 | Loss:4.229699821183176 | top1:54.292930603027344

Epoch: [5 | 4000] LR: 0.012400
34/34 Data:0.000 | Batch:0.357 | Total:0:00:14 | ETA:0:00:01 | Loss:2.1007380919022993 | top1:55.30303192138672

Epoch: [6 | 4000] LR: 0.013000
34/34 Data:0.000 | Batch:0.554 | Total:0:00:13 | ETA:0:00:01 | Loss:1.2369536125298701 | top1:56.56565475463867

Epoch: [7 | 4000] LR: 0.013600
34/34 Data:0.000 | Batch:0.397 | Tot

34/34 Data:0.001 | Batch:0.445 | Total:0:00:14 | ETA:0:00:01 | Loss:4.640108166318951 | top1:50.0

Epoch: [57 | 4000] LR: 0.040000
34/34 Data:0.002 | Batch:0.641 | Total:0:00:15 | ETA:0:00:01 | Loss:9.733251504825823 | top1:52.52525329589844

Epoch: [58 | 4000] LR: 0.040000
34/34 Data:0.000 | Batch:0.509 | Total:0:00:15 | ETA:0:00:01 | Loss:44.52497990926107 | top1:52.52525329589844

Epoch: [59 | 4000] LR: 0.040000
34/34 Data:0.001 | Batch:0.401 | Total:0:00:14 | ETA:0:00:01 | Loss:18.161128434267912 | top1:50.50505065917969

Epoch: [60 | 4000] LR: 0.040000
34/34 Data:0.002 | Batch:0.422 | Total:0:00:14 | ETA:0:00:01 | Loss:4.145705143610637 | top1:48.48484802246094

Epoch: [61 | 4000] LR: 0.040000
34/34 Data:0.002 | Batch:0.402 | Total:0:00:15 | ETA:0:00:01 | Loss:1.7477500799930457 | top1:52.77777862548828

Epoch: [62 | 4000] LR: 0.039999
34/34 Data:0.001 | Batch:0.457 | Total:0:00:16 | ETA:0:00:01 | Loss:0.7645670175552368 | top1:51.51515197753906

Epoch: [63 | 4000] LR: 0.039999
34

34/34 Data:0.000 | Batch:0.334 | Total:0:00:13 | ETA:0:00:01 | Loss:94.0048040043224 | top1:48.989898681640625

Epoch: [113 | 4000] LR: 0.039977
34/34 Data:0.000 | Batch:0.429 | Total:0:00:14 | ETA:0:00:01 | Loss:3.291678173975511 | top1:48.232322692871094

Epoch: [114 | 4000] LR: 0.039976
34/34 Data:0.000 | Batch:0.532 | Total:0:00:14 | ETA:0:00:01 | Loss:0.9577234441583807 | top1:50.0

Epoch: [115 | 4000] LR: 0.039976
34/34 Data:0.000 | Batch:0.392 | Total:0:00:14 | ETA:0:00:01 | Loss:2.0423104347604695 | top1:54.0404052734375

Epoch: [116 | 4000] LR: 0.039975
34/34 Data:0.000 | Batch:0.520 | Total:0:00:14 | ETA:0:00:01 | Loss:118.66008411754261 | top1:52.77777862548828

Epoch: [117 | 4000] LR: 0.039974
34/34 Data:0.002 | Batch:0.333 | Total:0:00:13 | ETA:0:00:01 | Loss:80.68019962310791 | top1:51.010101318359375

Epoch: [118 | 4000] LR: 0.039973
34/34 Data:0.001 | Batch:0.399 | Total:0:00:13 | ETA:0:00:01 | Loss:20.228982437740672 | top1:53.030303955078125

Epoch: [119 | 4000] LR: 0

...

34/34 Data:0.002 | Batch:0.252 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6308307467084943 | top1:64.14141082763672

Epoch: [2635 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.243 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6334047064636693 | top1:68.18181610107422

Epoch: [2636 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.250 | Total:0:00:09 | ETA:0:00:01 | Loss:0.6549938736539899 | top1:68.18181610107422

Epoch: [2637 | 4000] LR: 0.000001
34/34 Data:0.002 | Batch:0.251 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6343106510061206 | top1:66.41413879394531

Epoch: [2638 | 4000] LR: 0.000001
34/34 Data:0.001 | Batch:0.253 | Total:0:00:09 | ETA:0:00:01 | Loss:0.6296630005041758 | top1:65.1515121459961

Epoch: [2639 | 4000] LR: 0.000001
34/34 Data:0.001 | Batch:0.261 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6065128165664095 | top1:73.2323226928711

Epoch: [2640 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.252 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6249258743994164 | top1:68.43434143066406

Epoch: 

34/34 Data:0.000 | Batch:0.188 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6269816087953972 | top1:69.1919174194336

Epoch: [2690 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.207 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6426348613970208 | top1:66.16161346435547

Epoch: [2691 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.208 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6399626795089606 | top1:66.919189453125

Epoch: [2692 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.209 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6359259847438696 | top1:69.1919174194336

Epoch: [2693 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.217 | Total:0:00:07 | ETA:0:00:01 | Loss:0.626033528284593 | top1:68.6868667602539

Epoch: [2694 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.221 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6368715185107607 | top1:65.90908813476562

Epoch: [2695 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.221 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6368706903674386 | top1:63.38383865356445

Epoch: [269

34/34 Data:0.000 | Batch:0.209 | Total:0:00:07 | ETA:0:00:01 | Loss:0.650746793457956 | top1:63.13131332397461

Epoch: [2745 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.216 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6373142311067292 | top1:67.67676544189453

Epoch: [2746 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.203 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6285988733623967 | top1:67.17171478271484

Epoch: [2747 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.211 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6331206406607772 | top1:70.20201873779297

Epoch: [2748 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.222 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6332636367190968 | top1:69.1919174194336

Epoch: [2749 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.221 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6108170919346086 | top1:70.70706939697266

Epoch: [2750 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.220 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6426742763230295 | top1:65.40403747558594

Epoch: 

34/34 Data:0.000 | Batch:0.222 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6228264236088955 | top1:70.45454406738281

Epoch: [2800 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.216 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6320642567042148 | top1:69.44444274902344

Epoch: [2801 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.220 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6473900576432546 | top1:64.14141082763672
161/161 Data:0.000 | Batch:0.083 | Total:0:00:23 | ETA:0:00:00 | Loss:0.6105841797460276 | top1:68.5077896118164
39/39 Data:0.000 | Batch:0.132 | Total:0:00:08 | ETA:0:00:00 | Loss:0.4600091951015668 | top1:84.56410217285156

Epoch: [2802 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.224 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6347100427656462 | top1:66.919189453125

Epoch: [2803 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.210 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6496924685709404 | top1:63.13131332397461

Epoch: [2804 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.199 | Total:0:0

34/34 Data:0.000 | Batch:0.217 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6424517631530762 | top1:66.16161346435547

Epoch: [2853 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.220 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6429144740104675 | top1:62.878787994384766

Epoch: [2854 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.219 | Total:0:00:08 | ETA:0:00:01 | Loss:0.6430002595439102 | top1:65.40403747558594

Epoch: [2855 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.226 | Total:0:00:07 | ETA:0:00:01 | Loss:0.632339358329773 | top1:66.41413879394531

Epoch: [2856 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.234 | Total:0:00:07 | ETA:0:00:01 | Loss:0.629901485009627 | top1:68.18181610107422

Epoch: [2857 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.229 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6496766471501553 | top1:69.1919174194336

Epoch: [2858 | 4000] LR: 0.000001
34/34 Data:0.000 | Batch:0.210 | Total:0:00:07 | ETA:0:00:01 | Loss:0.6649190620942549 | top1:62.62626266479492

Epoch: 